# Codi per generar imatges *noisy*
Aquest codi en primer lloc carrega l'accés als fitxers de Google Drive.
En segon lloc, pren el nostre dataset d'imatges (del qual hem d'especificar la ruta) i itera sobre les dues subcarpetes que corresponen al conjunt d'entrenament i validació; aplicant una convolució amb un nucli gaussià i després afegint renou de tipus gaussià amb uns paràmetres donats. Després guarda les imatges en les carpetes *train_noisy* i *val_noisy*.

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import torch
import torch.nn.functional as F
from PIL import Image
import torchvision.transforms as transforms
import numpy as np
import cv2
import os

# Funció per crear el kernel gaussià
def gaussian_kernel(size, sigma):
    x = torch.arange(size).float() - (size - 1) / 2
    y = torch.arange(size).float() - (size - 1) / 2
    x, y = torch.meshgrid(x, y, indexing="ij")
    kernel = torch.exp(-0.5 * (x ** 2 + y ** 2) / sigma ** 2)
    kernel = kernel / kernel.sum()
    return kernel

# Funció per aplicar la convolució gaussiana
def apply_gaussian_blur(image, kernel_size=11, sigma=1.8):
    kernel = gaussian_kernel(kernel_size, sigma)
    kernel = kernel.unsqueeze(0).unsqueeze(0).repeat(3, 1, 1, 1)

    image_np = np.array(image).astype(np.float32)  # mantenim [0, 255]
    image_tensor = torch.from_numpy(image_np).permute(2, 0, 1).unsqueeze(0)

    blurred_image = F.conv2d(image_tensor, kernel, groups=3, padding=kernel_size // 2)

    return blurred_image.squeeze(0).permute(1, 2, 0).clamp(0, 255).byte().numpy()

# Funció per afegir renou gaussià
def add_gaussian_noise(image, mean=0, std=3):
    noise = np.random.normal(mean, std, image.shape).astype(np.int16)
    noisy_image = image.astype(np.int16) + noise
    noisy_image = np.clip(noisy_image, 0, 255).astype(np.uint8)
    return noisy_image

# Directori base del dataset
base_folder = '/content/drive/MyDrive/TFG/UIB_SINGLE_IMAGE'
folders = ['train','val']

# Iterar sobre les carpetes train i val
for folder in folders:
    original_folder = os.path.join(base_folder, folder)
    noisy_folder = os.path.join(base_folder, folder + '_noisy')

    if not os.path.exists(original_folder):
        print(f"La carpeta {original_folder} no existeix! Saltant...")
        continue

    if not os.path.exists(noisy_folder):
        os.makedirs(noisy_folder)
        print(f"Carpeta creada: {noisy_folder}")

    for filename in os.listdir(original_folder):
        image_path = os.path.join(original_folder, filename)
        noisy_image_path = os.path.join(noisy_folder, filename)

        image = Image.open(image_path).convert("RGB")
        blurred_image = apply_gaussian_blur(image, kernel_size=11, sigma=5.0)
        noisy_blurred_image = add_gaussian_noise(blurred_image, mean=0, std=7)

        cv2.imwrite(noisy_image_path, cv2.cvtColor(noisy_blurred_image, cv2.COLOR_RGB2BGR))
        print(f"Imatge processada i guardada: {noisy_image_path}")

print("Procés complet!")